In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col

In [2]:
spark = SparkSession.builder.appName("TaxiStreaming").getOrCreate()

lines = spark.readStream \
    .format("socket") \
    .option("host", "socket_stream") \
    .option("port", 9999) \
    .load()

gps_df = lines.select(
    split(col("value"), ",").getItem(0).alias("trip_id"),
    split(col("value"), ",").getItem(1).alias("timestamp"),
    split(col("value"), ",").getItem(2).alias("lat"),
    split(col("value"), ",").getItem(3).alias("lon")
)

query = gps_df.writeStream \
    .format("memory") \
    .queryName("live_view") \
    .start()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/11 10:51:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/11 10:51:07 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
26/01/11 10:51:07 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-46f5366f-f2e3-4c09-a1fd-9aa3335eb65e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
26/01/11 10:51:07 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Dat

In [8]:
# Select top 5 rows to see if data is flowing
spark.sql("SELECT * FROM live_view").show(5)

# Or count how many rows you have received so far
spark.sql("SELECT count(*) FROM live_view").show()

+-------------------+---------+----------+---------+
|            trip_id|timestamp|       lat|      lon|
+-------------------+---------+----------+---------+
|1372636858620000589| 20000589|1372636858|-8.618643|
|1372636858620000589| 20000589|1372636873|-8.618499|
|1372636858620000589| 20000589|1372636888|-8.620326|
|1372636858620000589| 20000589|1372636903|-8.622153|
|1372636858620000589| 20000589|1372636918|-8.623953|
+-------------------+---------+----------+---------+
only showing top 5 rows
+--------+
|count(1)|
+--------+
|     221|
+--------+

